In [1]:
import logging
import time
import os
import pickle

#from DataPrep import data_prep

import numpy as np
import matplotlib.pyplot as plt

#import tensorflow_datasets as tfds
import tensorflow as tf

# Import tf_text to load the ops used by the tokenizer saved model
#import tensorflow_text  # pylint: disable=unused-import
import pandas as pd
import numpy as np
import re
import seaborn as sns

from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model,  Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dropout, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Concatenate
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Attention
from tensorflow.keras.optimizers import Adam, Adagrad
from keras.losses import sparse_categorical_crossentropy
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings
import random

In [2]:
Cds_Pred = {}
fname_list = ["PichiaArch1","PichiaArch1_HumanContext","PichiaArch2","PichiaArch2_HumanContext"]
N_seq = len(fname_list)
for i in range(N_seq ):
    Cds_Pred[i] = pd.read_excel('./PredictedSequences/' + fname_list[i]+'_Lactoferrin.xlsx').iloc[:,2:-1].values.tolist()


In [3]:
def AA_Codon_list():
    dic_AA_codon = {'A': ['GCT', 'GCC', 'GCA', 'GCG'],
                    'C': ['TGT', 'TGC'],
                    'D': ['GAT', 'GAC'],
                    'E': ['GAA', 'GAG'],
                    'F': ['TTT', 'TTC'],
                    'G': ['GGT', 'GGA', 'GGC', 'GGG'],
                    'H': ['CAT', 'CAC'],
                    'I': ['ATT', 'ATC', 'ATA'],
                    'K': ['AAA', 'AAG'],
                    'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
                    'M': ['ATG'],
                    'N': ['AAT', 'AAC'],
                    'P': ['CCT', 'CCC', 'CCA', 'CCG'],
                    'Q': ['CAA', 'CAG'],
                    'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
                    'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
                    'T': ['ACT', 'ACC', 'ACA', 'ACG'],
                    'V': ['GTT', 'GTC', 'GTA', 'GTG'],
                    'W': ['TGG'],
                    'Y': ['TAT', 'TAC'],
                    '*': ['TAA', 'TAG', 'TGA']}

    codon_list = []
    AA_list = []
    for key in dic_AA_codon:
        for i in range(len(dic_AA_codon[key])):
            AA_list.append(key)
        for i in dic_AA_codon[key]:
            codon_list.append(i)
    return AA_list, codon_list

def de_tokenize_Codon(sequences):
    AA_list, codon_list = AA_Codon_list()
    keys = codon_list
    values = range(1, len(codon_list) + 1)
    Codon_dict = dict(zip(keys, values))
    seq_de_tokenized = []
    for s in range(len(sequences)):
        seq = sequences[s]
        temp = ''
        for i in range(len(seq)):
             temp = temp+keys[values.index(seq[i])]
        

        seq_de_tokenized.append(temp)

    return seq_de_tokenized, Codon_dict

In [4]:
Cds_seq_de_tokenized = {}
for i in range(N_seq):
    Cds_seq_de_tokenized[i], Cds_seq_tokenizer = de_tokenize_Codon(Cds_Pred[i])


In [5]:
pd.DataFrame(Cds_seq_de_tokenized).to_csv('DeTokenized_Lactoferrin_PredictedSeq.csv')

In [6]:
fasta_format = {}
for i in range(len(fname_list)):
    fasta_format[i] = ">Lactoferrin Homo Sapiens " + fname_list[i]+"\n"
    n_frags_60 = int(np.ceil(len(Cds_seq_de_tokenized[i][0])/60))
    for j in range (n_frags_60):
        fasta_format[i] = fasta_format[i] + Cds_seq_de_tokenized[i][0][j*60: (j+1)*60] + "\n"
    
    saveFasta = open(r'PredSeq_'+ fname_list[i]+'_DNA.fasta', 'w+')
    saveFasta.write(fasta_format[i])
    saveFasta.close()
    

In [7]:
len(fname_list)

4